In [1]:
import nltk as nl
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity, treebank, stopwords
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
import numpy as np
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize.treebank import TreebankWordDetokenizer

np.set_printoptions(threshold=sys.maxsize)

nl.download('punkt')
nl.download('stopwords')

stopwords = stopwords.words('english') #pythonspot tutorial
stop = set()

flag =1
inFile = open("./stoplist.txt","r")
while flag == 1:
    line = inFile.readline()
    if line == "":
        flag = 0
    stop.add(line[:-1])
inFile.close()

df = pd.read_csv('big_train.csv', delimiter=',') 

comments =[]
for i in range(len(df)):
    tokens = nl.regexp_tokenize(df.iloc[i, 1], r'[,\.\?\n\t0123456789 /\'><;:[\]{}+-=_()!@#$%^&*~`"]\s*', gaps=True)
    tokens = [token.lower() for token in tokens]
    reduced = set()
    for word in tokens:
        if word not in stop:
            reduced.add(word)
    un_token = TreebankWordDetokenizer()
    comments.append(un_token.detokenize(tokens))
    
count_vector = CountVectorizer()
dataframe = count_vector.fit_transform(comments)
bag_o_words = pd.DataFrame(dataframe.toarray(), columns = count_vector.get_feature_names_out())

print(bag_o_words)




[nltk_data] Downloading package punkt to
[nltk_data]     /Users/csilberlicht/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/csilberlicht/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


       aa  aaa  aaaannnnyyyywwwwhhhheeeerrrreeee  aac  aadministrators   
0       0    0                                 0    0                0  \
1       0    0                                 0    0                0   
2       0    0                                 0    0                0   
3       0    0                                 0    0                0   
4       0    0                                 0    0                0   
...    ..  ...                               ...  ...              ...   
14024   0    0                                 0    0                0   
14025   0    0                                 0    0                0   
14026   0    0                                 0    0                0   
14027   0    0                                 0    0                0   
14028   0    0                                 0    0                0   

       aagadu  aah  aalertbot  aang  aaot  ...  雲水  電視劇  顧家輝  飞天  飞天号航天服   
0           0    0          0     0

In [ ]:
    # reduced = set()
    # for word in tokens:
    #     if word not in stop:
    #         reduced.add(word)
    bag = bag_o_words.to_numpy()
    # bag_matrix = np.empty(0)
# for i, row in bag_o_words.iterrows():
#     val = df.loc[i, :].values.flatten().tolist()
#     bag_matrix=np.append(bag_matrix,val, axis=0)

# bag_matrix